##Union-Find Data Structure

Let's do our first data structure: Union-Find and use this with some of that sparse data matrix thing.

I guess for this one it's easiest to use object oriented programming

"In some applications we wish to know simply whether or not a vertex x is connected to a vertex y in a graph; the actual path connecting them may not be relevant. The efficient algorithms that have been developed are of independent interest because they can also be used for processing sets (collections of objects)."

"Graphs correspond to sets of objects in a natural way: vertices correspond to objects and edges mean 'is in the same set as.' Each connected component corresponds to a different set. For sets, we're interested in the fundamental question 'is x in the same set as y?' This clearly corresponds to the fundamental graph question 'is vertex x connected to vertex y?'"

"Our objective is to write a function that can check if two vertices x and y are in the same set (or, in the graph representation, the same connected component) and, if not, can put them in the same set (put an edge between them in the graph). From the correspondence with the set problem, the addition of a new edge is called a <i>union</i> operation and the queries are called <i>find</i> operations."

"Instead of building a direct adjacency-list or other representation of the graph, we'll gain efficiency by using an internal structure specifically oriented towards supporting the <i>union</i> and <i>find</i> operations. This internal structure will be a <i>forest of trees</i>, one for each connected component. We need to be able to find if out if two vertices belong to the same tree and to be able to combine two trees into one."

"It must be emphasized that the only relationship between these union-find trees and the underlying graph with the given edges is that they divide the vertices into sets in the same way. There is no correspondence between the paths that connect nodes in the tree and the paths that connect nodes in the graph."

One thing this means in practical purposes is that if two nodes are already connected, even if it is by a very long path, and if you add a new edge between them, i.e. union the two vertices, the union algorithm will do no extra work, it will simply notice that they are already connected and stop. 

In [1]:
class Graph(object):
    """Undirected Graph with Union-Find Data Structure"""
    
    def __init__(self, n):
        if(n < 1):
            raise IndexError("n must be at least one")
        if(type(n) != int):
            raise KeyError("n must be an integer")
        self.order = n
        self.points_to = [i for i in range(self.order)] #To keep track of the unions   
        
        #We won't be using the following two for union-find, but they're useful in general
        self.size = 0
        self.edges = {}
        
    def find(self, x):
        """Here we do find"""
        if(x >= self.order):
            raise IndexError("x must be less than n")
        out = x
        while(True):
            if(out == self.points_to[out]):
                break
            else:
                out = self.points_to[out]
        return out         
        
    def union(self, x, y, weight = 1):
        """Here we do union"""
        if(x >= self.order or y >= self.order):
            raise IndexError("x and y must be less than n")

        y_root = self.find(y)
        x_root = self.find(x)
        if(x_root != y_root):
            self.points_to[y_root] = x_root
        
            
        self.edges[(x,y)] = weight     
        self.edges[(y,x)] = weight
        self.size += weight
            
    def connected(self):
        """Checks to see if the graph is all connected"""
        out = True
        root = self.points_to[self.order - 1]
        for i in range(self.order - 1):
            if(self.find(i) != root):
                out = False
                break
        return out
        
        

G = Graph(5) 

G.union(0,1)
G.union(1,2)
G.union(2,3)

print(G.connected())

G.union(3,4)
G.union(4,0)

print(G.connected())
G.points_to

False
True


[0, 0, 0, 0, 0]

"The algorith described above has bad worst-case performance because the trees formed because the trees can be degenerate. " For example, just by switching the numbers as below, we can see that this " produces a long chain with 0 pointing to 1, 1 pointing to 2, etc."

In [2]:
G = Graph(5) 

G.union(1,0)
G.union(2,1)
G.union(3,2)

print(G.connected())

G.union(4,3)
#G.union(0,4)

print(G.connected())
G.points_to

False
True


[1, 2, 3, 4, 4]

"This kind of structure takes time proportional to $V^2$ to build, and has time proportional to V for an average equivalence test."

"Several methods have been suggested to deal with this problem. When a tree rooted at i is to be merged with a tree rooted at j, one of the nodes must remain a root and the other (and all its descendants) must go one level down the tree. To minimize the distance to the root for the most nodes, it makes sense to take as the root the node with more descendants. This idea is called <i>weight balancing</i>.

Also, "Ideally, we would like every node to point directly to the root of its tree.... We can approach the ideal by making all the nodes we do examine point to the root. This method is called <i>path compression</i>"

##QR Algorithm

"The basis for the QR method for calculating the eigenvalues of A is the fact that an $n \times n$ real matrix can be written as 

$$A = QR$$

where Q is orthogonal and R is upper triangular. (I'll probably need some of these definitions as well later). The method is efficient for the calculation of all eigenvalues of a matrix."

"The construction of Q and R proceeds as follows. Matrices $P_1, P_2, ... , P_{n-1}$ are constructed so that $P_{n-1}P_{n-2}\cdot \cdot \cdot P_2 P_1 A = R$ is upper triangular. These matrices can be chosen as orthogonal matrices and are called <i>householder matrices</i>. If we let 

$$Q^T = P_{n-1}P_{n-2}\cdot \cdot \cdot P_2 P_1$$

then we have $Q^T A = R$" and therefore $A = QR$ (because of orthogonality or something like that)

"We discuss the construction of the P's presently. First we state how the QR factorization of A is used to find eigenvalues of A. We define sequences of matrices $A_1, A_2, ..., A_m, ...; Q_1, Q_2, ..., Q_m, ... ; R_1, R_2, ..., R_m, ...$ by this process:

<b>Step 1.</b> Set $A_1 = A, Q_1 = Q$ and $R_1 = R$

<b>Step 2.</b> Set $A_2 = R_{1} Q_{1}$; then factor $A_2$ as $A_2 = Q_2 R_2$ (QR factorization of $A_2$)

<b>Step m.</b> Set $A_m = R_{m-1} Q_{m-1}$; then factor $A_m$ as $A_m = Q_m R_m$ (QR factorization of $A_m$)


"Matrix $A_m$ will tend toward a triangular or nearly triangular  form. Thus the eigenvalues will of $A_m$ will be easy to calculate",i.e., they're just the diagonal entries ". The importance is that if the eigenvalues can be ordered as $|\lambda_1|>|\lambda_2|> ... > |\lambda_n|>0$, then the following is true:

As m increases the eigenvalues of $A_m$ approach the eigenvalues of A.

The proof of this fact is well beyond the scope of this book.

Furthermore, "If A is symmetric, matrices $A_m$ converge to a diagonal matrix with the eigenvalues on the diagonal"

"Finally, after we find the eigenvalues of A, the corresponding eigenvectors can be found by solving $(\lambda \mathbb{I} - A) X = 0$, subject to some side condition such as $|X| = 1$"



From http://www.math.tamu.edu/~dallen/linear_algebra/chpt6.pdf

"The idea in QR factorization is to first find $P_1$ which, when multiplied on the left of A, will produce zeros belos $a_{11}$. That is, we want

$$P_1 \left(\begin{matrix} 
a_{11} \ a_{12} \ ... \ a_{1n} \\
a_{21} \ a_{22} \ ... \ a_{2n} \\
\vdots \ ~~~ \ddots \ ~~~ \vdots \\
a_{n1} \ a_{n2} \ ... \ a_{nn}\\
\end{matrix}\right)
=
 \left(\begin{array} 
\tilde {a_{11}} \ ~~ \tilde {a_{12}} \ ... \ \tilde {a_{1n}} \\
\textbf{0} \ ~~ \tilde {a_{22}} \ ... \ \tilde {a_{2n}} \\
\vdots \ ~~~~~ \ddots \ ~~~~~ \vdots \\
\textbf{0} \ ~~ \tilde {a_{n2}} \ ... \ \tilde {a_{nn}}\\
\end{array}\right)
$$

After this is done, we find $P_2$ which will produce


$$P_2 P_1 A = P_2  \left(\begin{matrix} 
\tilde {a_{11}} \ ~~ \tilde {a_{12}} \ ... \ \tilde {a_{1n}} \\
\textbf{0} \ ~~ \tilde {a_{22}} \ ... \ \tilde {a_{2n}} \\
\vdots \ ~~~~~ \ddots \ ~~~~~ \vdots \\
\textbf{0} \ ~~ \tilde {a_{n2}} \ ... \ \tilde {a_{nn}}\\
\end{matrix}\right)=
 \left(\begin{matrix} 
\hat {a_{11}} \ ~ \hat {a_{12}} ~ \hat {a_{13}} \ ... \ \hat {a_{1n}} \\
\textbf{0} \ ~ \hat {a_{22}} ~ \hat {a_{23}} \ ... \ \hat {a_{2n}} \\
\textbf{0} \ ~~ \textbf {0} ~~~~~ \hat {a_{33}}  \ ... \ ~~ \hat {a_{3n}} \\
\vdots \ ~~~~ \ddots \ ~~~~~ \vdots \\
\textbf{0} \ ~~~~~ \textbf{0} \ ~~ \hat {a_{3n}} \ ... \ \hat {a_{nn}}\\
\end{matrix}\right)
$$

.